In [13]:
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk

In [14]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [15]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        pass

In [16]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)

In [17]:
import random
random_data = random.sample(review_data,len(review_data))

In [18]:
reviewText = []
for i in range(len(random_data)):
    reviewText.append(random_data[i]["reviewText"])

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1,2), lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(reviewText)

In [20]:
y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
y_data=np.array(y_data)

In [21]:
from sklearn import cross_validation

In [22]:
skf = cross_validation.StratifiedKFold(y_data, n_folds=10,random_state=500)

In [23]:
for train_index, test_index in skf:
    X_train, X_test = D[train_index], D[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

In [24]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((83868, 3585351), (83868,), (9316, 3585351), (9316,))

BaggingClassifier

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
result.predict(X_test)
print("Bagging :",accuracy_score(result.predict(X_test),y_test))

RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
seed = 7
num_trees = 100
max_features = 3
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
result.predict(X_test)
print("RandomForest :",accuracy_score(result.predict(X_test),y_test))

ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

seed = 7
num_trees = 100
max_features = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
result.predict(X_test)
print("ExtraTrees :",accuracy_score(result.predict(X_test),y_test))

AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 30
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
result.predict(X_test)
print("AdaBoost :",accuracy_score(result.predict(X_test),y_test))

GradientBoostingClassifier

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier
seed = 7
num_trees = 100
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
result.predict(X_test)
print("GradientBoosting :",accuracy_score(result.predict(X_test),y_test))